In [18]:
from parquet_data_reader import ParquetDataReader
import polars as pl
import pandas as pd
import numpy as np
from utils.process_data import process_item_data
pl.Config.set_tbl_cols(-1)

polars.config.Config

In [21]:
article_df = pl.read_parquet("../generated_data/train/items_processed.parquet")
article_df.head()

article_id,last_modified_time,premium,published_time,article_type,category,total_inviews,total_pageviews,total_read_time,sentiment_score,subcategory_1,subcategory_2,subcategory_3,subcategory_4,subcategory_5,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,vector_9,vector_10,vector_11,vector_12,vector_13,vector_14,vector_15,vector_16,vector_17,vector_18,vector_19,vector_20,vector_21,vector_22,vector_23,vector_24,vector_25,vector_26,vector_27,vector_28,vector_29,vector_30,vector_31,vector_32,vector_33,vector_34,vector_35,vector_36,vector_37,vector_38,vector_39,vector_40,vector_41,vector_42,vector_43,vector_44,vector_45,vector_46,vector_47,vector_48,vector_49,vector_50,vector_51,vector_52,vector_53,vector_54,vector_55,vector_56,vector_57,vector_58,vector_59,vector_60,vector_61,vector_62,vector_63,vector_64,vector_65,vector_66,vector_67,vector_68,vector_69,vector_70,vector_71,vector_72,vector_73,vector_74,vector_75,vector_76,vector_77,vector_78,vector_79,vector_80,vector_81,vector_82,vector_83,vector_84,vector_85,vector_86,vector_87,vector_88,vector_89,vector_90,vector_91,vector_92,vector_93,vector_94,vector_95,vector_96,vector_97,vector_98,vector_99,vector_100,vector_101,vector_102,vector_103,vector_104,vector_105,vector_106,vector_107,vector_108,vector_109,vector_110,vector_111,vector_112,vector_113,vector_114,vector_115,vector_116,vector_117,vector_118,vector_119,vector_120,vector_121,vector_122,vector_123,vector_124,vector_125,vector_126,vector_127,vector_128,vector_129,vector_130,vector_131,vector_132,vector_133,vector_134,vector_135,vector_136,vector_137,vector_138,vector_139,vector_140,vector_141,vector_142,vector_143,vector_144,vector_145,vector_146,vector_147,vector_148,vector_149,vector_150,vector_151,vector_152,vector_153,vector_154,vector_155,vector_156,vector_157,vector_158,vector_159,vector_160,vector_161,vector_162,vector_163,vector_164,vector_165,vector_166,vector_167,vector_168,vector_169,vector_170,vector_171,vector_172,vector_173,vector_174,vector_175,vector_176,vector_177,vector_178,vector_179,vector_180,vector_181,vector_182,vector_183,vector_184,vector_185,vector_186,vector_187,vector_188,vector_189,vector_190,vector_191,vector_192,vector_193,vector_194,vector_195,vector_196,vector_197,vector_198,vector_199,vector_200,vector_201,vector_202,vector_203,vector_204,vector_205,vector_206,vector_207,vector_208,vector_209,vector_210,vector_211,vector_212,vector_213,vector_214,vector_215,vector_216,vector_217,vector_218,vector_219,vector_220,vector_221,vector_222,vector_223,vector_224,vector_225,vector_226,vector_227,vector_228,vector_229,vector_230,vector_231,vector_232,vector_233,vector_234,vector_235,vector_236,vector_237,vector_238,vector_239,vector_240,vector_241,vector_242,vector_243,vector_244,vector_245,vector_246,vector_247,vector_248,vector_249,vector_250,vector_251,vector_252,vector_253,vector_254,vector_255,vector_256,vector_257,vector_258,vector_259,vector_260,vector_261,vector_262,vector_263,vector_264,vector_265,vector_266,vector_267,vector_268,vector_269,vector_270,vector_271,vector_272,vector_273,vector_274,vector_275,vector_276,vector_277,vector_278,vector_279,vector_280,vector_281,vector_282,vector_283,vector_284,vector_285,vector_286,vector_287,vector_288,vector_289,vector_290,vector_291,vector_292,vector_293,vector_294,vector_295,vector_296,vector_297,vector_298,vector_299,vector_300,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11
i32,i64,bool,i64,u32,i16,i32,i32,f32,f32,i64,i64,i64,i64,i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f3